In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
from textstat import textstat

# Initialize NLTK's sentiment intensity analyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Read the input Excel file
input_df = pd.read_excel('Input.xlsx')

# Create empty lists to store analysis results
positive_scores = []
negative_scores = []
polarity_scores = []
subjectivity_scores = []
avg_sentence_lengths = []
percentage_complex_words = []
fog_indices = []
avg_words_per_sentence = []
complex_word_counts = []
word_counts = []
syllables_per_word = []
personal_pronouns_counts = []
avg_word_lengths = []

# Loop through each row in the DataFrame and process the URLs
for index, row in input_df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    # Send a GET request to the URL
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract article title and text
        article_title = soup.find('h1').text.strip() if soup.find('h1') else "Title not found"
        article_text_element = soup.find('div', class_='article-body')
        
        if article_text_element:
            article_text = article_text_element.get_text(separator='\n').strip()
        else:
            article_text = "Text not found"

        # Perform sentiment analysis
        sentiment_scores = sia.polarity_scores(article_text)
        positive_scores.append(sentiment_scores['pos'])
        negative_scores.append(sentiment_scores['neg'])
        polarity_scores.append(sentiment_scores['compound'])

        # Calculate other text statistics
        sentences = sent_tokenize(article_text)
        avg_sentence_length = sum(len(word_tokenize(sent)) for sent in sentences) / len(sentences)
        avg_sentence_lengths.append(avg_sentence_length)

        percentage_complex_words.append(textstat.difficult_words(article_text) / textstat.lexicon_count(article_text) * 100)
        fog_indices.append(textstat.gunning_fog(article_text))
        avg_words_per_sentence.append(textstat.avg_sentence_length(article_text))
        complex_word_counts.append(textstat.difficult_words(article_text))
        word_counts.append(textstat.lexicon_count(article_text))
        syllables_per_word.append(textstat.syllable_count(article_text) / textstat.lexicon_count(article_text) if textstat.lexicon_count(article_text) > 0 else 0)

        # Count personal pronouns
        personal_pronouns = ['I', 'me', 'my', 'mine', 'myself', 'you', 'your', 'yours', 'yourself', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'we', 'us', 'our', 'ours', 'ourselves', 'they', 'them', 'their', 'theirs', 'themselves']
        personal_pronouns_count = sum(article_text.lower().count(word) for word in personal_pronouns)
        personal_pronouns_counts.append(personal_pronouns_count)

        # Calculate average word length
        words = word_tokenize(article_text)
        avg_word_length = sum(len(word) for word in words) / len(words)
        avg_word_lengths.append(avg_word_length)

        # Save the extracted article text to a text file
        with open(f'{url_id}.txt', 'w', encoding='utf-8') as file:
            file.write(article_title + '\n\n')
            file.write(article_text)

    else:
        print(f"Failed to fetch data from {url}")
        # Fill in default values for missing data
        positive_scores.append(0)
        negative_scores.append(0)
        polarity_scores.append(0)
        avg_sentence_lengths.append(0)
        percentage_complex_words.append(0)
        fog_indices.append(0)
        avg_words_per_sentence.append(0)
        complex_word_counts.append(0)
        word_counts.append(0)
        syllables_per_word.append(0)
        personal_pronouns_counts.append(0)
        avg_word_lengths.append(0)

# Create a DataFrame to store the analysis results
output_df = pd.DataFrame({
    'URL_ID': input_df['URL_ID'],
    'URL': input_df['URL'],
    'POSITIVE SCORE': positive_scores,
    'NEGATIVE SCORE': negative_scores,
    'POLARITY SCORE': polarity_scores,
    'SUBJECTIVITY SCORE': subjectivity_scores,
    'AVG SENTENCE LENGTH': avg_sentence_lengths,
    'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
    'FOG INDEX': fog_indices,
    'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
    'COMPLEX WORD COUNT': complex_word_counts,
    'WORD COUNT': word_counts,
    'SYLLABLE PER WORD': syllables_per_word,
    'PERSONAL PRONOUNS': personal_pronouns_counts,
    'AVG WORD LENGTH': avg_word_lengths
})

# Add these print statements to identify the issue
print(f"URL_ID length: {len(input_df['URL_ID'])}")
print(f"URL length: {len(input_df['URL'])}")
print(f"Positive Scores length: {len(positive_scores)}")
# ... Add similar print statements for other lists

# Create a DataFrame to store the analysis results
output_df = pd.DataFrame({
    'URL_ID': input_df['URL_ID'],
    'URL': input_df['URL'],
    'POSITIVE SCORE': positive_scores,
    'NEGATIVE SCORE': negative_scores,
    # ... Include other columns here
})


# Save the output DataFrame to an Excel file
output_df.to_excel('Output.xlsx', index=False)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Failed to fetch data from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to fetch data from https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
